In [ ]:
import numpy as np
from time import clock
from stratigraphy import stratal as strat

import pycpt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

In [ ]:
strati = strat.stratal(filename='input-60.yml')

print("Number of cpus used: ",strati.nbCPUs)
print("Number of stratigraphic layers created: ",strati.layNb)

In [ ]:
t = clock()
strati.readStratalData()
print('Reading stratal outputfile (%0.02f seconds)'% (clock() - t))

In [ ]:
t = clock()
strati.buildLonLatMesh(res=0.2, nghb=3)
print(' ')
print('Building stratal mesh (%0.02f seconds)'% (clock() - t))

In [ ]:
topocmap = pycpt.load.cmap_from_cptcity_url('gmt/GMT_globe.cpt')
depocmap = pycpt.load.cmap_from_cptcity_url('cmocean/balance.cpt')

In [ ]:
def plotElevation( data, cmin, cmax, colormap):
    '''
    data: dataset to plot
    cmin,cmax: extent of the colormap
    colormap: color sca\le to use   
    '''
    
    # Figure size is defined here
    fig = plt.figure(1, figsize=(9,9))
    
    ax = plt.gca()
    im = ax.imshow(np.flipud(data), interpolation='nearest', cmap=colormap,
                     vmin=cmin, vmax=cmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(im,cax=cax)
    plt.tight_layout()

    plt.show()
    plt.close(fig)

In [ ]:
plotElevation( strati.zi[:,:,-1], -10000, 10350, topocmap)

In [ ]:
plotElevation( strati.edi[:,:,-1], -200, 200, depocmap)

In [ ]:
plotElevation( strati.zi[800:1000,1200:1500,-1], -10000, 10350, topocmap)

In [ ]:
plotElevation( strati.edi[900:1000,1300:1400,-1], -200, 200, depocmap)

In [ ]:
t = clock()
strati.writeMesh(vtkfile='amazon', lats=[800,1000], lons=[1200,1500], hscale=11000.)
print('Writing stratal mesh (%0.02f seconds)'% (clock() - t))

In [ ]:
from pyevtk.hl import gridToVTK
from scipy.ndimage import gaussian_filter

def lonlat2xyz(lon, lat, elev):
    
    radius = 6378137.+elev
    lon_r = np.radians(lon)
    lat_r = np.radians(lat)

    xs =  radius * np.cos(lat_r) * np.cos(lon_r)
    ys = radius * np.cos(lat_r) * np.sin(lon_r)
    zs = radius * np.sin(lat_r) 
    
    return xs, ys, zs
    
def writesMesh(vtkfile='mesh', lons=None, lats=None, sigma=1.):
    
    ilon = np.linspace(0., 360., strati.nx)
    ilat = np.linspace(0., 180., strati.ny)

    if lons is None:
        lons = [0,strati.nx]
    if lats is None:
        lats = [0,strati.ny]

    x = np.zeros((lons[1]-lons[0], lats[1]-lats[0], strati.layNb))
    y = np.zeros((lons[1]-lons[0], lats[1]-lats[0], strati.layNb))
    z = np.zeros((lons[1]-lons[0], lats[1]-lats[0]))
    sz = np.zeros((lons[1]-lons[0], lats[1]-lats[0], strati.layNb))
    e = np.zeros((lons[1]-lons[0], lats[1]-lats[0], strati.layNb))
    h = np.zeros((lons[1]-lons[0], lats[1]-lats[0], strati.layNb))
    t = np.zeros((lons[1]-lons[0], lats[1]-lats[0], strati.layNb))

    zz = strati.zi[lats[0]:lats[1],lons[0]:lons[1],-1]
    zz = gaussian_filter(zz, sigma)
    
    import pandas as pd
    import gc
    
    for k in range(strati.layNb-1,-1,-1):
        print(k)
        th = gaussian_filter(strati.thi[:,:,k], sigma)
        th[th<0] = 0.
        if k < strati.layNb-1:
            thu = gaussian_filter(strati.thi[:,:,k+1], sigma)
            thu[thu<0] = 0.
            
        for j in range(lats[1]-lats[0]):
            for i in range(lons[1]-lons[0]):
                
                lx = ilon[i+lons[0]]
                ly = ilat[j+lats[0]]
                
                if k == strati.layNb-1:
                    z[i,j] = zz[j,i]
                else:
                    z[i,j] -= thu[j+lats[0],i+lons[0]]
                
                x[i,j,k],y[i,j,k],sz[i,j,k] = lonlat2xyz(lx-180.,ly-90.,z[i,j])
                
                e[i,j,k] = strati.zi[j+lats[0],i+lons[0],k]
                h[i,j,k] = th[j+lats[0],i+lons[0]]
                t[i,j,k] = k

                
    del ilon,ilat,z
    gc.collect()
    np.savez_compressed('dataset', x=x, y=y, z=sz, e=e, h=h, t=t)
#     gridToVTK(vtkfile, x, y, sz) #, pointData = {"dep elev" : e, "th" :h, "layID" :t})

    return

In [ ]:
writesMesh(vtkfile='sp')

In [1]:
import numpy as np
from pyevtk.hl import gridToVTK

vtkfile='sphere'
loaded = np.load("dataset.npz")
x = loaded['x']
y = loaded['y']
z = loaded['z']
e = loaded['e']
h = loaded['h']
t = loaded['t']

In [2]:
gridToVTK(vtkfile, x, y, z, pointData = {"dep elev" : e, "th" :h, "layID" :t})

'/live/share/global/globalData/sphere.vts'